<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/cnes/indicadores_monitor_rosa_cnes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação
Este notebook foi desenvolvido como parte do projeto Monitor Rosa, um conjunto de ferramentas, softwares e outros artefatos cujo principal objetivo é melhorar o cenário de diagnósticos de câncer de mama no Brasil, idealmente melhorando diminuindo a proporção de diagnósticos tardios/diagnósticos precoces.

Os arquivos utilizados neste notebook foram coletados através dos scripts presentes [neste repositório](https://github.com/heber-augusto/devops-pysus-get-files). Os arquivos foram armazenados em um bucket do Google Storage no formato parquet compactados (gzip).

O principal objetivo deste notebook é consolidar os arquivos coletados do ftp do SUS em bases/arquivos que facilitem análises e utilização dos dados.

Os arquivos criados por este notebook possuem apenas os dados de SP, estão salvos em uma pasta chamada "consolidado" (no mesmo bucket do google onde estão salvos os arquivos coletados do FTP do SUS) e são organizados em: 

 - pacientes.parquet.gzip: informações consolidadas de cada paciente (detalhes [neste comando](#scrollTo=N9Kzt8iI24Iw));
 - cancer.parquet.gzip: informações consolidadas dos dois tipos de procedimentos - quimioterapia e radioterapia (detalhes [neste comando](#scrollTo=iMkYq9UGIhP6));
 - cancer_aq.parquet.gzip: registros de quimioterapia consolidados em um único arquivo (criado a partir [deste comando](#scrollTo=bCCfeKAdMM0y));
 - cancer_ar.parquet.gzip: registros de radioterapia consolidados em um único arquivo (criado a partir [deste comando](#scrollTo=bCCfeKAdMM0y));
 - procedimentos_e_pacientes.parquet.gzip: Procedimentos com dados consolidados de pacientes (criado a partir [deste comando](#scrollTo=6zSk7STZdbVI));
 - dados_estad_municipio_mensal.parquet.gzip: dados consolidados por mês/municipio/estadiamento (criado a partir [deste comando](#scrollTo=N65hubVP_--P));
 - dados_estad_mensal.parquet.gzip: dados consolidados por mês/estadiamento (criado a partir [deste comando](#scrollTo=zta5pk3Oj4Vj)).
 



# Instalação de bibliotecas e pacotes para leitura de arquivos

In [1]:
import glob
import os
from google.colab import auth

## Bibliotecas utilizadas na análise exploratória

In [2]:
import pandas as pd
import numpy as np 

## Configurações iniciais para conectar com bucket no Google Storage
 - Autenticação do Google Colab
 - Definição do nome do projeto

 Para a execução dos comandos desta seção, o arquio gcp-leitura.json deve ser inserido na raiz do colab.

In [3]:
# auth.authenticate_user()

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

# estado contendo consolidação dos dados
state = 'SP'


## Definições globais

In [4]:

DESTINATION_FOLDER = os.path.join(
    r'/content/',
    local_folder_name,
    project_folder_name, 
    state)
DESTINATION_FOLDER = f'{DESTINATION_FOLDER}/consolidado/'
DESTINATION_FOLDER

'/content/colab/monitor/SP/consolidado/'

## Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [5]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  57619      0 --:--:-- --:--:-- --:--:-- 57619
OK
23 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.12_amd64.deb ...
Unpacking gcsfuse (0.41.12) ...
Setting up gcsfuse (0.41.12) ...


## Nome/caminho do arquivo json contendo credenciais de acesso ao storage do Google

In [7]:
service_account_json = '/content/gcp-leitura.json'

## Montagem do bucket em uma pasta local do google colab

In [8]:
!mkdir {local_folder_name}
!gcsfuse --key-file {service_account_json} --implicit-dirs {bucket_id} {local_folder_name}

2023/02/22 12:28:49.372602 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/colab
2023/02/22 12:28:49.387447 Opening GCS connection...
2023/02/22 12:28:50.412947 Mounting file system "observatorio-oncologia"...
2023/02/22 12:28:50.413775 File system has been successfully mounted.


## Função para facilitar coleta de arquivo

In [9]:
def get_files(state, year, month, file_type, file_group):
    initial_path = os.path.join(r'/content/',local_folder_name,project_folder_name)
    internal_folder = f"""{state}/{year}/{month}/{file_type}/{file_group}"""
    # print(f"{initial_path}/{internal_folder}/*.parquet.gzip")
    return glob.glob(f"{initial_path}/{internal_folder}/*.parquet.gzip")    

In [18]:
arquivos = get_files('SP', 2021, 11, 'CNES', 'EQ')
eq_test = pd.read_parquet(arquivos[0])
eq_test[(eq_test.TIPEQUIP == '1') & ((eq_test.CODEQUIP == '02') | (eq_test.CODEQUIP == '03'))]

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TIPEQUIP,CODEQUIP,QT_EXIST,QT_USO,IND_SUS,IND_NSUS,COMPETEN,NAT_JUR
30,2035766,350010,0209,,,,M,3,01558435000119,1,...,,,1,02,1,1,0,1,202111,2240
80,3415082,350010,0209,,,,M,3,05808046000155,1,...,,,1,02,1,1,0,1,202111,2240
150,6219926,350010,0209,,,,M,3,10688485000101,1,...,,,1,02,1,1,1,0,202111,2062
593,2774720,350070,0206,,,,M,3,43138320000115,1,...,,,1,02,1,1,1,0,202111,3999
1124,2028077,350160,0212,,,,M,3,01061386000104,1,...,,,1,02,1,1,1,0,202111,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200402,7987293,355700,,,,,M,3,23604288000109,1,...,,,1,02,1,1,1,0,202111,2062
200564,2055546,355710,0215,,,,M,3,56364037000234,1,...,,,1,02,1,1,0,1,202111,2062
200611,2081377,355710,0215,,,,E,3,72957814000120,1,...,,,1,02,1,1,1,0,202111,3999
200615,2084309,355710,0215,,,,M,3,53807475000584,1,...,,,1,02,1,1,0,1,202111,2143


# SIA PA, AQ e AR: Leitura, filtro e transformação inicial dos arquivos

## Informações sobre filtros pertinentes ao contexto de câncer de mama:

### SIH: realizar filtro através da variável DIAG_PRINC (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### RHC: realizar filtro através da variável LOCTUDET (3 caracteres)
 * Filtro: C50

### SIA – APAC de Quimioterapia e Radioterapia (AQ e AR)
Realizar filtro através da variável AP_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### SIA – Procedimentos ambulatoriais (PA)

Os arquivos de procedimentos ambulatoriais são um pouco diferentes por um motivo: a pessoa já pode ter o diagnóstico e está realizando um procedimento OU a pessoa está realizando um exame com finalidade diagnóstica (mamografia, ultrassonografia, etc). Então, neste caso, podemos pensar em dois filtros:

Realizar filtro através da variável PA_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

Realizar filtro através da variável do código de procedimento ambulatorial “PA_PROC_ID” (10 caracteres)
 * Filtros:
  * 201010569	BIOPSIA/EXERESE DE NÓDULO DE MAMA
  * 201010585	PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA
  * 201010607	PUNÇÃO DE MAMA POR AGULHA GROSSA
  * 203010035	EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)
  * 203010043	EXAME CITOPATOLOGICO DE MAMA
  * 203020065	EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA
  * 203020073	EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA
  * 205020097	ULTRASSONOGRAFIA MAMARIA BILATERAL
  * 208090037	CINTILOGRAFIA DE MAMA (BILATERAL)
  * 204030030	MAMOGRAFIA
  * 204030188	MAMOGRAFIA BILATERAL PARA RASTREAMENTO

## Variáveis de filtro

In [ ]:
# filtro pelo cid
cid_filter = ['C500', 'C501', 'C502', 'C503', 'C504', 'C505', 'C506', 'C508', 'C509']

# dicionario de procedimentos
proc_id_dict = {
    '0201010569': 'BIOPSIA/EXERESE DE NÓDULO DE MAMA',
    '0201010585': 'PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA',
    '0201010607': 'PUNÇÃO DE MAMA POR AGULHA GROSSA',
    '0203010035': 'EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)',
    '0203010043': 'EXAME CITOPATOLOGICO DE MAMA',
    '0203020065': 'EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA',
    '0203020073': 'EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA',
    '0205020097': 'ULTRASSONOGRAFIA MAMARIA BILATERAL',
    '0208090037': 'CINTILOGRAFIA DE MAMA (BILATERAL)',
    '0204030030': 'MAMOGRAFIA',
    '0204030188': 'MAMOGRAFIA BILATERAL PARA RASTREAMENTO'
    }
proc_id_filter = list(proc_id_dict.keys())


## Funções de filtro para arquivo SIA PA, AQ e AR

In [19]:
def filter_pa_content(df):
    """

    """
    return df[df.PA_CIDPRI.isin(cid_filter) & \
              df.PA_PROC_ID.isin(proc_id_filter)]

def filter_ar_content(df):
    """

    """
    return df[df.AP_CIDPRI.isin(cid_filter)]

filter_aq_content = filter_ar_content


def filter_eq_content(df):
    """

    """
    return df[(df.TIPEQUIP == '1') & ((df.CODEQUIP == '02') | (df.CODEQUIP == '03'))]    






## Função para unir diversos arquivos em um único datraframe

In [20]:
def create_cancer_dataframe(file_paths, filter_function=filter_pa_content):
    """

    """
    filtered_contents = [
      filter_function(pd.read_parquet(file_path))
      for file_path in file_paths
      ]

    return pd.concat(
        filtered_contents, 
        ignore_index=True)



## Função para retornar lista de arquivos (caminho completo)

In [21]:
def get_file_paths(states, years, months, file_type, file_group):
    """

    """
    file_paths = []
    for state in states:
        for year in years:
            for month in months:
                file_paths.extend(
                    get_files(
                        state,
                        year,
                        month,
                        file_type,
                        file_group)
                )
    return file_paths

## SIA PA: Leitura e união de dados para o período desejado




### Estado, anos e meses a serem lidos e processados

In [25]:
states = [state,]
start_year = 2021
end_year = 2021
years  = [f'{year + start_year:02d}' for year in range(end_year - start_year + 1)]
months = [f'{month + 1:02d}' for month in range(12)]
file_type = 'CNES'

### Monta lista de arquivos a serem lidos

In [26]:
file_paths_by_type = {}

# Arquivos de produção ambulatorial
file_paths_by_type['EQ'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'EQ'
)
file_paths_by_type

{'EQ': ['/content/colab/monitor/SP/2021/01/CNES/EQ/EQSP2101.parquet.gzip',
  '/content/colab/monitor/SP/2021/02/CNES/EQ/EQSP2102.parquet.gzip',
  '/content/colab/monitor/SP/2021/03/CNES/EQ/EQSP2103.parquet.gzip',
  '/content/colab/monitor/SP/2021/04/CNES/EQ/EQSP2104.parquet.gzip',
  '/content/colab/monitor/SP/2021/05/CNES/EQ/EQSP2105.parquet.gzip',
  '/content/colab/monitor/SP/2021/06/CNES/EQ/EQSP2106.parquet.gzip',
  '/content/colab/monitor/SP/2021/07/CNES/EQ/EQSP2107.parquet.gzip',
  '/content/colab/monitor/SP/2021/08/CNES/EQ/EQSP2108.parquet.gzip',
  '/content/colab/monitor/SP/2021/09/CNES/EQ/EQSP2109.parquet.gzip',
  '/content/colab/monitor/SP/2021/10/CNES/EQ/EQSP2110.parquet.gzip',
  '/content/colab/monitor/SP/2021/11/CNES/EQ/EQSP2111.parquet.gzip',
  '/content/colab/monitor/SP/2021/12/CNES/EQ/EQSP2112.parquet.gzip']}

In [27]:
df = create_cancer_dataframe(
    file_paths_by_type['EQ'], 
    filter_function=filter_eq_content)

df

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TIPEQUIP,CODEQUIP,QT_EXIST,QT_USO,IND_SUS,IND_NSUS,COMPETEN,NAT_JUR
0,2035766,350010,0209,,,,M,3,01558435000119,1,...,,,1,02,1,1,0,1,202101,2240
1,3415082,350010,0209,,,,M,3,05808046000155,1,...,,,1,02,1,1,0,1,202101,2240
2,6219926,350010,0209,,,,M,3,10688485000101,1,...,,,1,02,1,1,1,0,202101,2062
3,2774720,350070,0206,,,,M,3,43138320000115,1,...,,,1,02,1,1,1,0,202101,3999
4,2028077,350160,0212,,,,M,3,01061386000104,1,...,,,1,02,1,1,1,0,202101,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13544,7987293,355700,,,,,M,3,23604288000109,1,...,,,1,02,1,1,1,0,202112,2062
13545,2055546,355710,0215,,,,M,3,56364037000234,1,...,,,1,02,1,1,0,1,202112,2062
13546,2081377,355710,0215,,,,E,3,72957814000120,1,...,,,1,02,1,1,1,0,202112,3999
13547,2084309,355710,0215,,,,M,3,53807475000584,1,...,,,1,02,1,1,0,1,202112,2143


In [ ]:
file_paths_by_type = {}

# Arquivos de produção ambulatorial
file_paths_by_type['PA'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'PA'
)

# Arquivos de radioterapia
file_paths_by_type['AR'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'AR'
)

# Arquivos de quimioteraia
file_paths_by_type['AQ'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'AQ'
)

In [ ]:
print(f"""Identificados {len(file_paths_by_type['PA'])} arquivos de produção ambulatorial""")
print(f"""Identificados {len(file_paths_by_type['AR'])} arquivos de radioterapia""")
print(f"""Identificados {len(file_paths_by_type['AQ'])} arquivos de quimioterapia""")

Identificados 317 arquivos de produção ambulatorial
Identificados 170 arquivos de radioterapia
Identificados 170 arquivos de quimioterapia


# Montagem do dataset cancer AR e AQ

Estes dois datasets (cancer_aq e cancer AR) não realizam nenhum tipo de filtro ou transformação nos dados originais, apenas consolidam os registros de todos os anos/meses, coletados do FTP do SUS, em um único dataframe.



## Cria um único dataframe a partir dos conteúdos filtrados
**ESTE PASSO PODE SER PULADO CASO OS ARQUIVOS JÁ ESTEJAM CRIADOS. NESTE CASO, SEGUIR PARA LEITURA ([NESTE COMANDO](#scrollTo=01d35Inz8hju))**

In [ ]:
cancer_dataframe_pa = create_cancer_dataframe(file_paths_by_type['PA'], filter_function=filter_pa_content)
cancer_dataframe_aq = create_cancer_dataframe(file_paths_by_type['AQ'], filter_function=filter_ar_content)
cancer_dataframe_ar = create_cancer_dataframe(file_paths_by_type['AR'], filter_function=filter_aq_content)

## Salva arquivos para acelerar próximas execuções do notebook

In [ ]:
cancer_dataframe_pa.to_parquet(
    'cancer_pa.parquet.gzip', 
    compression='gzip')

cancer_dataframe_aq.to_parquet(
    'cancer_aq.parquet.gzip', 
    compression='gzip')

cancer_dataframe_ar.to_parquet(
    'cancer_ar.parquet.gzip', 
    compression='gzip')

## Dataframe de PA

In [ ]:
cancer_dataframe_pa

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2081377,350000,EP,355710,0000,0000,0000,05,61,I,...,1,K,0,,NaN,NaN,NaN,NaN,NaN,NaN
1,2081377,350000,EP,355710,0000,0000,0000,05,61,I,...,1,K,0,,NaN,NaN,NaN,NaN,NaN,NaN
2,2081377,350000,EP,355710,0000,0000,0000,05,61,I,...,1,K,0,,NaN,NaN,NaN,NaN,NaN,NaN
3,2081377,350000,EP,355710,0000,0000,0000,05,61,I,...,1,K,0,,NaN,NaN,NaN,NaN,NaN,NaN
4,2081377,350000,EP,355710,0000,0000,0000,05,61,I,...,1,K,0,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368989,2027240,355030,PG,355030,0000,0000,0000,05,00,M,...,1,K,0,,0.0,0.0,0.0,121002,,1031
368990,2027240,355030,PG,355030,0000,0000,0000,05,00,M,...,1,K,0,,0.0,0.0,0.0,121002,,1031
368991,2027240,355030,PG,355030,0000,0000,0000,05,00,M,...,1,K,0,,0.0,0.0,0.0,121002,,1031
368992,2027240,355030,PG,355030,0000,0000,0000,05,00,M,...,1,K,0,,0.0,0.0,0.0,121002,,1031


In [ ]:
cancer_dataframe_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368994 entries, 0 to 368993
Data columns (total 60 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PA_CODUNI   368994 non-null  object
 1   PA_GESTAO   368994 non-null  object
 2   PA_CONDIC   368994 non-null  object
 3   PA_UFMUN    368994 non-null  object
 4   PA_REGCT    368994 non-null  object
 5   PA_INCOUT   368994 non-null  object
 6   PA_INCURG   368994 non-null  object
 7   PA_TPUPS    368994 non-null  object
 8   PA_TIPPRE   368994 non-null  object
 9   PA_MN_IND   368994 non-null  object
 10  PA_CNPJCPF  368994 non-null  object
 11  PA_CNPJMNT  368994 non-null  object
 12  PA_CNPJ_CC  368994 non-null  object
 13  PA_MVM      368994 non-null  object
 14  PA_CMP      368994 non-null  object
 15  PA_PROC_ID  368994 non-null  object
 16  PA_TPFIN    368994 non-null  object
 17  PA_SUBFIN   368994 non-null  object
 18  PA_NIVCPL   368994 non-null  object
 19  PA_DOCORIG  368994 non-

## Dataframe de AQ

In [ ]:
cancer_dataframe_aq

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,AQ_DTINI2,AQ_CIDINI3,AQ_DTINI3,AQ_CONTTR,AQ_DTINTR,AQ_ESQU_P1,AQ_TOTMPL,AQ_TOTMAU,AQ_ESQU_P2,AP_NATJUR
0,200801,EP,350000,2025507,3508205574489,200801,0304050130,213.4,352900,05,...,,,,S,20070801,TAMOX,060,060,,NaN
1,200801,EP,350000,2025507,3508205574500,200801,0304050040,79.75,352900,05,...,20061101,C500,20070101,S,20070606,TAMOX,060,060,,NaN
2,200801,EP,350000,2025507,3508205574621,200801,0304020354,147.1,352900,05,...,19960301,C508,20040901,S,20070101,EXAME,060,060,,NaN
3,200801,EP,350000,2773171,3508203923345,200801,0304020141,2828.4,353060,39,...,,,,N,20080111,ESQUE,008,000,,NaN
4,200801,EP,350000,2773171,3508204190227,200801,0304020338,301.5,353060,39,...,,,,N,20070213,ESQUE,012,010,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5415650,202202,EP,350000,2077396,3522222582505,202201,0304020435,2149.5,354980,05,...,,,,S,20220128,CARBO,005,000,"PLATINA, D",3069
5415651,202202,EP,350000,2688689,3521272063815,202111,0304020435,0.0,355030,05,...,20190903,C504,20200205,S,20211123,PERTU,005,000,/TRASTU,3999
5415652,202202,EP,350000,2705982,3522218907922,202202,0304020435,1700.0,351620,05,...,,,,N,20220101,Docet,010,000,axelHercPe,3069
5415653,202202,EP,350000,2079798,3522217105429,202202,0304020443,34.1,350950,05,...,,,,S,20210318,HERCE,017,003,PTIN,1112


In [ ]:
cancer_dataframe_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5415655 entries, 0 to 5415654
Data columns (total 64 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   AP_MVM      object
 1   AP_CONDIC   object
 2   AP_GESTAO   object
 3   AP_CODUNI   object
 4   AP_AUTORIZ  object
 5   AP_CMP      object
 6   AP_PRIPAL   object
 7   AP_VL_AP    object
 8   AP_UFMUN    object
 9   AP_TPUPS    object
 10  AP_TIPPRE   object
 11  AP_MN_IND   object
 12  AP_CNPJCPF  object
 13  AP_CNPJMNT  object
 14  AP_CNSPCN   object
 15  AP_COIDADE  object
 16  AP_NUIDADE  object
 17  AP_SEXO     object
 18  AP_RACACOR  object
 19  AP_MUNPCN   object
 20  AP_UFNACIO  object
 21  AP_CEPPCN   object
 22  AP_UFDIF    object
 23  AP_MNDIF    object
 24  AP_DTINIC   object
 25  AP_DTFIM    object
 26  AP_TPATEN   object
 27  AP_TPAPAC   object
 28  AP_MOTSAI   object
 29  AP_OBITO    object
 30  AP_ENCERR   object
 31  AP_PERMAN   object
 32  AP_ALTA     object
 33  AP_TRANSF   object
 34  AP_DTOCOR   object

## Dataframe de AR

In [ ]:
cancer_dataframe_ar

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,AR_NUMC1,AR_INIAR1,AR_INIAR2,AR_INIAR3,AR_FIMAR1,AR_FIMAR2,AR_FIMAR3,AR_NUMC2,AR_NUMC3,AP_NATJUR
0,200801,EP,350000,2790556,3508205036259,200801,0304010090,940.68,350600,05,...,84,20080102,,,20080331,,,,,NaN
1,200801,EP,350000,2748223,3508205171460,200801,0304010294,247.92,350750,05,...,60,20080128,,,20080331,,,,,NaN
2,200801,EP,350000,2083086,3508205087112,200801,0304010090,335.24,352530,07,...,002,20080122,,,20080331,,,000,000,NaN
3,200801,EP,350000,2090236,3508211775740,200801,0304010286,1057.72,350550,07,...,003,20071220,,,20080529,,,,,NaN
4,200801,PG,350320,2082527,3507229450164,200801,0304010286,2179.92,350320,05,...,04,20080101,,,20080331,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190814,202202,PG,354340,2080400,3522226642946,202202,0304010413,5904.0,354340,05,...,,20220112,,,20220331,,,,,3999
190815,202202,PG,354870,2025361,3522230090764,202202,0304010413,5904.0,354870,05,...,,20220210,,,20220430,,,,,1244
190816,202202,PG,355030,2077590,3522221932493,202202,0304010413,5904.0,355030,07,...,,20220201,,,20220430,,,,,3999
190817,202202,PG,355030,2077590,3522221932559,202202,0304010413,5904.0,355030,07,...,,20220201,,,20220430,,,,,3999


In [ ]:
cancer_dataframe_ar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190819 entries, 0 to 190818
Data columns (total 74 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   AP_MVM      190819 non-null  object
 1   AP_CONDIC   190819 non-null  object
 2   AP_GESTAO   190819 non-null  object
 3   AP_CODUNI   190819 non-null  object
 4   AP_AUTORIZ  190819 non-null  object
 5   AP_CMP      190819 non-null  object
 6   AP_PRIPAL   190819 non-null  object
 7   AP_VL_AP    190819 non-null  object
 8   AP_UFMUN    190819 non-null  object
 9   AP_TPUPS    190819 non-null  object
 10  AP_TIPPRE   190819 non-null  object
 11  AP_MN_IND   190819 non-null  object
 12  AP_CNPJCPF  190819 non-null  object
 13  AP_CNPJMNT  190819 non-null  object
 14  AP_CNSPCN   190819 non-null  object
 15  AP_COIDADE  190819 non-null  object
 16  AP_NUIDADE  190819 non-null  object
 17  AP_SEXO     190819 non-null  object
 18  AP_RACACOR  190819 non-null  object
 19  AP_MUNPCN   190819 non-

## Leitura de arquivos (se já existirem)

In [ ]:
cancer_dataframe_ar = pd.read_parquet(
    f'{DESTINATION_FOLDER}cancer_ar.parquet.gzip')

cancer_dataframe_ar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190819 entries, 0 to 190818
Data columns (total 74 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   AP_MVM      190819 non-null  object
 1   AP_CONDIC   190819 non-null  object
 2   AP_GESTAO   190819 non-null  object
 3   AP_CODUNI   190819 non-null  object
 4   AP_AUTORIZ  190819 non-null  object
 5   AP_CMP      190819 non-null  object
 6   AP_PRIPAL   190819 non-null  object
 7   AP_VL_AP    190819 non-null  object
 8   AP_UFMUN    190819 non-null  object
 9   AP_TPUPS    190819 non-null  object
 10  AP_TIPPRE   190819 non-null  object
 11  AP_MN_IND   190819 non-null  object
 12  AP_CNPJCPF  190819 non-null  object
 13  AP_CNPJMNT  190819 non-null  object
 14  AP_CNSPCN   190819 non-null  object
 15  AP_COIDADE  190819 non-null  object
 16  AP_NUIDADE  190819 non-null  object
 17  AP_SEXO     190819 non-null  object
 18  AP_RACACOR  190819 non-null  object
 19  AP_MUNPCN   190819 non-

In [ ]:
cancer_dataframe_aq = pd.read_parquet(
    f'{DESTINATION_FOLDER}cancer_aq.parquet.gzip')

cancer_dataframe_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5415655 entries, 0 to 5415654
Data columns (total 64 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   AP_MVM      object
 1   AP_CONDIC   object
 2   AP_GESTAO   object
 3   AP_CODUNI   object
 4   AP_AUTORIZ  object
 5   AP_CMP      object
 6   AP_PRIPAL   object
 7   AP_VL_AP    object
 8   AP_UFMUN    object
 9   AP_TPUPS    object
 10  AP_TIPPRE   object
 11  AP_MN_IND   object
 12  AP_CNPJCPF  object
 13  AP_CNPJMNT  object
 14  AP_CNSPCN   object
 15  AP_COIDADE  object
 16  AP_NUIDADE  object
 17  AP_SEXO     object
 18  AP_RACACOR  object
 19  AP_MUNPCN   object
 20  AP_UFNACIO  object
 21  AP_CEPPCN   object
 22  AP_UFDIF    object
 23  AP_MNDIF    object
 24  AP_DTINIC   object
 25  AP_DTFIM    object
 26  AP_TPATEN   object
 27  AP_TPAPAC   object
 28  AP_MOTSAI   object
 29  AP_OBITO    object
 30  AP_ENCERR   object
 31  AP_PERMAN   object
 32  AP_ALTA     object
 33  AP_TRANSF   object
 34  AP_DTOCOR   object

In [ ]:
cancer_dataframe_pa = pd.read_parquet(
    f'{DESTINATION_FOLDER}cancer_pa.parquet.gzip')

cancer_dataframe_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368994 entries, 0 to 368993
Data columns (total 60 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PA_CODUNI   368994 non-null  object
 1   PA_GESTAO   368994 non-null  object
 2   PA_CONDIC   368994 non-null  object
 3   PA_UFMUN    368994 non-null  object
 4   PA_REGCT    368994 non-null  object
 5   PA_INCOUT   368994 non-null  object
 6   PA_INCURG   368994 non-null  object
 7   PA_TPUPS    368994 non-null  object
 8   PA_TIPPRE   368994 non-null  object
 9   PA_MN_IND   368994 non-null  object
 10  PA_CNPJCPF  368994 non-null  object
 11  PA_CNPJMNT  368994 non-null  object
 12  PA_CNPJ_CC  368994 non-null  object
 13  PA_MVM      368994 non-null  object
 14  PA_CMP      368994 non-null  object
 15  PA_PROC_ID  368994 non-null  object
 16  PA_TPFIN    368994 non-null  object
 17  PA_SUBFIN   368994 non-null  object
 18  PA_NIVCPL   368994 non-null  object
 19  PA_DOCORIG  368994 non-

In [ ]:
cancer_dataframe_ar.head()

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,AR_NUMC1,AR_INIAR1,AR_INIAR2,AR_INIAR3,AR_FIMAR1,AR_FIMAR2,AR_FIMAR3,AR_NUMC2,AR_NUMC3,AP_NATJUR
0,200801,EP,350000,2790556,3508205036259,200801,0304010090,940.68,350600,05,...,84,20080102,,,20080331,,,,,None
1,200801,EP,350000,2748223,3508205171460,200801,0304010294,247.92,350750,05,...,60,20080128,,,20080331,,,,,None
2,200801,EP,350000,2083086,3508205087112,200801,0304010090,335.24,352530,07,...,002,20080122,,,20080331,,,000,000,None
3,200801,EP,350000,2090236,3508211775740,200801,0304010286,1057.72,350550,07,...,003,20071220,,,20080529,,,,,None
4,200801,PG,350320,2082527,3507229450164,200801,0304010286,2179.92,350320,05,...,04,20080101,,,20080331,,,,,None


## Conteúdo das colunas com datas

In [ ]:
cancer_dataframe_ar[['AP_CMP','AP_DTINIC', 'AP_DTFIM', 'AP_DTOCOR']]

,AP_CMP,AP_DTINIC,AP_DTFIM,AP_DTOCOR
0,200801,20080102,20080331,20080128
1,200801,20080128,20080331,
2,200801,20080122,20080331,
3,200801,20080102,20080331,
4,200801,20080101,20080331,
...,...,...,...,...
190814,202202,20220101,20220331,20220221
190815,202202,20220210,20220430,20220225
190816,202202,20220201,20220430,20220211
190817,202202,20220201,20220430,20220211


# Montagem do dataset cancer, consolidando procedimentos

Este dataset consolida registros dois tipos de procedimentos (AQ e AR)

Possui as seguintes colunas:

 - data: valores da coluna **AP_CMP** (AR e AQ);
 - paciente (cns_encrypted): valores da coluna **AP_CNSPCN** (AR e AQ);
 - estadiamento: valores dascolunas **AQ_ESTADI** (AQ) e **AR_ESTADI** (AR);
 - custo: valores da coluna **AP_VL_AP** (AQ ou AR) convertido para double;
 - municipio: valores da coluna **AP_MUNPCN** (AQ ou AR);
 - obito: valores da coluna **AP_OBITO** (AQ ou AR), convertido para inteiro;
 - tipo: tipo de procedimento ('ar' ou 'aq').

## Transformação dos tipos das colunas 
 - custo (AP_VL_AP) em double
 - Indicação de óbito (AP_OBITO) em inteiro.

In [ ]:
cancer_dataframe_aq['custo'] = cancer_dataframe_aq['AP_VL_AP'].astype(np.double)
cancer_dataframe_ar['custo'] = cancer_dataframe_ar['AP_VL_AP'].astype(np.double)
cancer_dataframe_aq['obito'] = cancer_dataframe_aq['AP_OBITO'].astype(np.integer)
cancer_dataframe_ar['obito'] = cancer_dataframe_ar['AP_OBITO'].astype(np.integer)

## União entre AQ e AR

In [ ]:
columns_aq = ['AP_CMP', 'AP_CNSPCN', 'AQ_ESTADI', 'custo', 'AP_MUNPCN', 'obito']
columns_ar = ['AP_CMP', 'AP_CNSPCN', 'AR_ESTADI', 'custo', 'AP_MUNPCN', 'obito']

normalized_columns = ['data','paciente','estadiamento', 'custo', 'municipio', 'obito']

renamed_aq = cancer_dataframe_aq[columns_aq]
renamed_aq.columns = normalized_columns
renamed_aq['tipo'] = 'aq'

renamed_ar = cancer_dataframe_ar[columns_ar]
renamed_ar.columns = normalized_columns
renamed_ar['tipo'] = 'ar'

cancer_dataframe = pd.concat(
    [
      renamed_aq, 
      renamed_ar
    ], 
    ignore_index=True)
cancer_dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,data,paciente,estadiamento,custo,municipio,obito,tipo
0,200801,{{||}|~,1,213.40,352900,0,aq
1,200801,{{{{{{,1,79.75,352900,0,aq
2,200801,{{{{~~~,4,147.10,351730,0,aq
3,200801,{{|{~||~,4,2828.40,353060,0,aq
4,200801,{{{~{|},1,301.50,353060,0,aq
...,...,...,...,...,...,...,...
5606469,202202,{{{{~~}{,2,5904.00,354340,0,ar
5606470,202202,{{~|,2,5904.00,354870,0,ar
5606471,202202,{{{||||,1,5904.00,355030,0,ar
5606472,202202,{{|{|~},0,5904.00,355030,0,ar


## Cria arquivo de registros de procedimentos (radioterapia e quimioterapia)

In [ ]:
cancer_dataframe.to_parquet(
    f'cancer.parquet.gzip', 
    compression='gzip')

In [ ]:
cancer_dataframe = pd.read_parquet(
    f'{DESTINATION_FOLDER}cancer.parquet.gzip')

cancer_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606474 entries, 0 to 5606473
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   data          object 
 1   paciente      object 
 2   estadiamento  object 
 3   custo         float64
 4   municipio     object 
 5   obito         int64  
 6   tipo          object 
dtypes: float64(1), int64(1), object(5)
memory usage: 299.4+ MB


In [ ]:
cancer_dataframe

,data,paciente,estadiamento,custo,municipio,obito,tipo
0,200801,{{||}|~,1,213.40,352900,0,aq
1,200801,{{{{{{,1,79.75,352900,0,aq
2,200801,{{{{~~~,4,147.10,351730,0,aq
3,200801,{{|{~||~,4,2828.40,353060,0,aq
4,200801,{{{~{|},1,301.50,353060,0,aq
...,...,...,...,...,...,...,...
5606469,202202,{{{{~~}{,2,5904.00,354340,0,ar
5606470,202202,{{~|,2,5904.00,354870,0,ar
5606471,202202,{{{||||,1,5904.00,355030,0,ar
5606472,202202,{{|{|~},0,5904.00,355030,0,ar


# Montagem do dataset de Exames de Paciente (1 linha por paciente)

Colunas:

 - Chave da paciente (cns_encrypted)
 - Custo total do tratamento
 - Estadiamento inicial
 - Estadiamento final
 - Indicação de óbito
 - Localização de moradia

## Proposta de solução:

 - Tipos de arquivos a serem utilizados: AQ e AR
 - chave do paciente:
  - AQ: coluna AP_CNSPCN
  - AR: coluna AP_CNSPCN  
 - custo total do tratamento: será uma estimativa considerando apenas radioterapia e quimioterapia, calculado somando os valores dos procedimentos (presentes em AR e AQ), para cada chave de paciente. Sendo:
   - custos em AQ: soma de AP_VL_AP;
   - custos em AR: soma de AP_VL_AP.
 - Estadiamento inicial: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais antigo de radioterapia ou quimioterapia, de um determinado paciente;
 - Estadiamento final: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais recente de radioterapia ou quimioterapia, de um determinado paciente;
 - Localização de moradia: utilizar coluna AP_MUNPCN (presentes em AR e AQ). Talvez criar dois campos, AP_MUNPCN presente no registro mais antigo e AP_MUNPCN presente no registro mais novo;
 - Indicação de óbito: valor máximo do campo AP_OBITO, presente em AQ e AR (0: sem indicação e 1:com indicação de óbito) O valor 1 indicará óbito.

## Cálculo dos valores considerando apenas AQ e AR

**SE O ARQUIVO DE PACIENTES JA FOI CRIADO, PASSAR PARA [ESTE COMANDO](#scrollTo=N65hubVP_--P)**

In [ ]:
df_paciente = cancer_dataframe\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['paciente'])\
    .agg(
        data_primeiro_estadiamento=('data', 'first'),         
        data_ultimo_estadiamento=('data', 'last'),                  
        primeiro_estadiamento=('estadiamento', 'first'), 
        maior_estadiamento=('estadiamento', 'max'),
        ultimo_estadiamento=('estadiamento', 'last'),
        custo_total=('custo', 'sum'),
        primeiro_municipio=('municipio', 'first'), 
        ultimo_municipio=('municipio', 'last'),
        indicacao_obito=('obito', 'max'),
        )\
    .reset_index()
   
df_paciente

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,898 0000 8725 6,201107,201108,2,2,2,3928.00,355220,355220,0
1,8980500671835 8,200903,200905,2,2,2,239.25,354890,354890,0
2,{{}{{{{{|,201004,201006,3,3,3,239.25,352900,352900,0
3,{}{|}{{|,200910,201105,1,1,1,4283.69,353660,353660,0
4,{}{~~~||{{{~,201102,201104,3,3,3,2400.00,510030,510030,0
...,...,...,...,...,...,...,...,...,...,...
180789,{{|}~},201709,202202,2,2,2,5430.50,355030,355030,0
180790,{{}{{~|,201305,201805,1,1,1,4785.00,350950,350950,0
180791,||{{{{|~,201004,201006,2,2,2,2171.96,280300,280300,0
180792,{|{{{,201206,201207,0,0,0,3420.00,355030,355030,0


## Validação dados cálculos em um dos pacientes

In [ ]:
cancer_dataframe[cancer_dataframe.paciente == df_paciente['paciente'][65794]][['data','paciente','estadiamento','municipio', 'obito']]

,data,paciente,estadiamento,municipio,obito
4506550,202004,{{}}}{|},4,352500,0
4564082,202005,{{}}}{|},4,352500,0
4590120,202006,{{}}}{|},4,352500,0
4643350,202007,{{}}}{|},4,352500,0
4674151,202008,{{}}}{|},4,352500,0
4733963,202009,{{}}}{|},4,352500,0
4787851,202011,{{}}}{|},2,352500,0
4798464,202010,{{}}}{|},2,352500,0
4841334,202012,{{}}}{|},2,352500,0
4886107,202101,{{}}}{|},2,352500,0


## Pacientes com marcação de óbito

In [ ]:
df_paciente[df_paciente.indicacao_obito == 1]

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
63,|{{~~{{{,201808,202010,3,3,3,2153.25,355560,355560,1
76,|{{~{{{,201112,201603,2,3,2,60593.65,354990,354990,1
80,|{{|~|{{{,200908,202003,3,3,2,34069.50,352900,352900,1
88,|{{{{~{{{~,200808,201005,3,3,3,6145.70,351670,351670,1
155,|{||}{~{{{,201001,201405,3,4,4,47220.24,314810,314810,1
...,...,...,...,...,...,...,...,...,...,...
180608,{{|{{}{},201108,201202,0,3,3,5600.00,354780,355030,1
180611,{{|{{}}{~,201408,202006,2,4,4,18351.35,355030,355030,1
180615,{{|{{~|{,201303,201412,4,4,4,23182.60,355030,355030,1
180746,{{|}{|}|,201101,201509,3,4,4,32370.55,352500,352500,1


## Pacientes com ultimo estadiamento menor que o maior estadiamento identificado

In [ ]:
df_paciente[df_paciente.maior_estadiamento > df_paciente.ultimo_estadiamento]

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
15,|{{|{{{,201207,201712,0,3,0,6769.50,355030,355030,0
45,|{{~~|{{{,200912,201406,1,1,0,4147.00,354990,354990,0
47,|{{~~{{{{,201006,201501,2,2,1,16281.00,354750,353360,0
49,|{{~~}~{{{,201007,201601,2,3,2,7691.55,354150,354150,0
53,|{{~~|}|~{{{,200905,201401,3,3,0,14394.80,350330,350330,0
...,...,...,...,...,...,...,...,...,...,...
180756,{{|}}|,201302,201606,2,4,3,1116.50,355030,355030,0
180757,{{|}~~{|,201610,202002,1,3,2,7035.50,355030,355030,0
180762,{{|},201601,201611,4,4,0,13053.00,354530,354530,0
180775,{{|}~{|{,201507,202108,3,4,3,14648.00,355030,355030,0


## Cria arquivo de pacientes

In [ ]:
df_paciente.to_parquet(
    f'pacientes.parquet.gzip', 
    compression='gzip')

## Le arquivo de pacientes, se já existir

In [ ]:
df_paciente = pd.read_parquet(
    f'{DESTINATION_FOLDER}pacientes.parquet.gzip')

In [ ]:
df_paciente

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,898 0000 8725 6,201107,201108,2,2,2,3928.00,355220,355220,0
1,8980500671835 8,200903,200905,2,2,2,239.25,354890,354890,0
2,{{}{{{{{|,201004,201006,3,3,3,239.25,352900,352900,0
3,{}{|}{{|,200910,201105,1,1,1,4283.69,353660,353660,0
4,{}{~~~||{{{~,201102,201104,3,3,3,2400.00,510030,510030,0
...,...,...,...,...,...,...,...,...,...,...
180789,{{|}~},201709,202202,2,2,2,5430.50,355030,355030,0
180790,{{}{{~|,201305,201805,1,1,1,4785.00,350950,350950,0
180791,||{{{{|~,201004,201006,2,2,2,2171.96,280300,280300,0
180792,{|{{{,201206,201207,0,0,0,3420.00,355030,355030,0


# Montagem do dataset de Procedimentos com dados consolidados Paciente (1 linha por procedimento)

Colunas:

  - data: data do procedimento;
  - paciente: código do paciente; 
  - estadiamento: estadiamento anotado no procedimento;
  - custo: custo do procedimento;
  - municipio: municipio do estadiamento;
  - obito: anotação de óbito no procedimento;
  - tipo: tipo de procedimento (ar - radioterapia ou aq - quimioterapia)
  **dados consolidados de todos os procedimentos do paciente**
  - data_primeiro_estadiamento: data do primeiro procedimento do paciente;
  - data_ultimo_estadiamento: data do último procedimento do paciente;
  - primeiro_estadiamento: primeiro estadiamento identificado no paciente;
  - maior_estadiamento: maior estadiamento identificado no paciente;
  - ultimo_estadiamento: ultimo estadiamento identificado no paciente;
  - custo_total: custo total dos procedimentos do paciente;
  - primeiro_municipio: municipio do primeiro procedimento do paciente;
  - ultimo_municipio: municipio do último procedimento do paciente;
  - indicacao_obito: indica se o paciente tem marcação de obito em algum procedimento.

In [ ]:
procedimentos_e_pacientes = pd.merge(
    cancer_dataframe, 
    df_paciente,
    on='paciente',
    how="outer")

In [ ]:
procedimentos_e_pacientes

,data,paciente,estadiamento,custo,municipio,obito,tipo,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,200801,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
1,200802,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
2,200803,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
3,200804,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
4,200805,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5606469,202202,{{{~{|,1,5904.0,355030,0,ar,202202,202202,1,1,1,5904.00,355030,355030,0
5606470,202201,}{~|}|{{{,1,5904.0,355220,0,ar,202201,202201,1,1,1,5904.00,355220,355220,0
5606471,202201,{{|}~~|},1,5904.0,350570,0,ar,202201,202201,1,1,1,5904.00,350570,350570,0
5606472,202201,{{|~|~~~,1,5904.0,355030,0,ar,202201,202201,1,1,1,5904.00,355030,355030,0


In [ ]:
procedimentos_e_pacientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5606474 entries, 0 to 5606473
Data columns (total 16 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   data                        object 
 1   paciente                    object 
 2   estadiamento                object 
 3   custo                       float64
 4   municipio                   object 
 5   obito                       int64  
 6   tipo                        object 
 7   data_primeiro_estadiamento  object 
 8   data_ultimo_estadiamento    object 
 9   primeiro_estadiamento       object 
 10  maior_estadiamento          object 
 11  ultimo_estadiamento         object 
 12  custo_total                 float64
 13  primeiro_municipio          object 
 14  ultimo_municipio            object 
 15  indicacao_obito             int64  
dtypes: float64(2), int64(2), object(12)
memory usage: 727.2+ MB


In [ ]:
procedimentos_e_pacientes.to_parquet(
    f'procedimentos_e_pacientes.parquet.gzip', 
    compression='gzip')

## Le arquivo de procedimentos e pacientes, se já existir

In [ ]:
procedimentos_e_pacientes = pd.read_parquet(
    f'{DESTINATION_FOLDER}procedimentos_e_pacientes.parquet.gzip')

# Criação de dataset de consolidando dados de pacientes por estadiamento/municipio/mês

Colunas:

  - data: mês do procedimento;
  - municipio: municipio do procedimento;
  - primeiro_estadiamento: estadiamento no primeiro procedimento do paciente (teoricamente, informa diagnostico precoce ou tardio);
  - custo_estadiamento: soma dos custos dos procedimentos para um dado mes/municipio/estadiamento;
  - numero_pacientes: número de pacientes no mes/municipio/estadiamento;
  - obtitos: número de obitos no mes/municipio/estadiamento;
  - obito_futuro: pacientes com anotação de obito no futuro;
  - numero_procedimentos: número de procedimentos realizados no mes/municipio/estadiamento;  

In [ ]:
dados_estad_municipio_mensal = procedimentos_e_pacientes\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['data', 'municipio','primeiro_estadiamento'])\
    .agg(
        custo_estadiamento=('custo', 'sum'),
        numero_pacientes=('paciente', 'nunique'), 
        obtitos=('obito', 'sum'),
        obito_futuro=('indicacao_obito', 'sum'),
        numero_procedimentos=('data','count')
        )\
    .reset_index()  
dados_estad_municipio_mensal

,data,municipio,primeiro_estadiamento,custo_estadiamento,numero_pacientes,obtitos,obito_futuro,numero_procedimentos
0,200801,110002,1,213.40,1,0,0,1
1,200801,110002,2,2458.65,2,0,0,2
2,200801,110004,1,79.75,1,0,0,1
3,200801,110004,2,239.25,3,0,0,3
4,200801,110004,3,4837.55,3,0,1,3
...,...,...,...,...,...,...,...,...
452487,202202,530010,4,79.75,1,0,0,1
452488,202202,530140,3,79.75,1,0,0,1
452489,202202,530170,3,1479.75,2,0,0,2
452490,202202,530180,1,79.75,1,0,0,1


In [ ]:
dados_estad_municipio_mensal.to_parquet(
    f'dados_estad_municipio_mensal.parquet.gzip', 
    compression='gzip')

## Le arquivo, se já existir

In [ ]:
dados_estad_municipio_mensal = pd.read_parquet(
    f'{DESTINATION_FOLDER}dados_estad_municipio_mensal.parquet.gzip')

# Criação de dataset de consolidando dados de pacientes por estadiamento/mês

Colunas:

  - data: mês do procedimento;
  - primeiro_estadiamento: estadiamento no primeiro procedimento do paciente (teoricamente, informa diagnostico precoce ou tardio);
  - numero_municipios: municipios com procedimentos mes/estadiamento;  
  - custo_estadiamento: soma dos custos dos procedimentos para um dado mes/estadiamento;
  - numero_pacientes: número de pacientes no mes/estadiamento;
  - obtitos: número de obitos no mes/estadiamento;
  - obito_futuro: pacientes com anotação de obito no futuro;
  - numero_procedimentos: número de procedimentos realizados no mes/estadiamento.  

In [ ]:
dados_estad_mensal = dados_estad_municipio_mensal\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['data', 'primeiro_estadiamento'])\
    .agg(
        custo_estadiamento=('custo_estadiamento', 'sum'),
        numero_pacientes=('numero_pacientes', 'sum'),         
        numero_municipios=('municipio', 'nunique'), 
        obtitos=('obtitos', 'sum'),
        obito_futuro=('obito_futuro', 'sum'),
        numero_procedimentos=('numero_procedimentos','sum')
        )\
    .reset_index()  
dados_estad_mensal

,data,primeiro_estadiamento,custo_estadiamento,numero_pacientes,numero_municipios,obtitos,obito_futuro,numero_procedimentos
0,200801,,5827.19,9,6,0,0,10
1,200801,0,507640.39,1102,182,2,47,1156
2,200801,1,813635.21,4713,443,0,88,4775
3,200801,2,1327561.51,7340,638,2,221,7449
4,200801,3,2040023.03,4378,568,1,314,4482
...,...,...,...,...,...,...,...,...
902,202202,0,601536.80,1537,187,2,2,1571
903,202202,1,1376737.15,7097,602,3,3,7222
904,202202,2,3683270.00,11360,648,7,7,11643
905,202202,3,5019041.55,10278,723,16,17,10575


In [ ]:
dados_estad_mensal.to_parquet(
    f'dados_estad_mensal.parquet.gzip', 
    compression='gzip')